In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!apt-get install -y chromium-browser

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.4 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,002 kB]
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,241 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-u

In [2]:
!pip install streamlit selenium beautifulsoup4 pandas requests lxml
!pip install pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [3]:
!which chromedriver
!chromium-browser --version

/usr/bin/chromedriver

Command '/usr/bin/chromium-browser' requires the chromium snap to be installed.
Please install it with:

snap install chromium



In [4]:
%%writefile enhanced_sports_search_app.py
import streamlit as st
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import re
from urllib.parse import quote, urljoin
import threading
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import warnings
warnings.filterwarnings('ignore')

# Custom CSS for better UI
def load_css():
    st.markdown("""
    <style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
    }
    .sport-selector {
        font-size: 1.2rem;
        margin-bottom: 1rem;
    }
    .player-card {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
        padding: 2rem;
        border-radius: 15px;
        margin-bottom: 1.5rem;
        border-left: 6px solid #1f77b4;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .stat-box {
        background-color: #ffffff;
        padding: 1.5rem;
        border-radius: 10px;
        margin: 0.8rem 0;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        border: 1px solid #e0e0e0;
    }
    .cricket-theme {
        border-left-color: #28a745;
        background: linear-gradient(135deg, #d4edda 0%, #c3e6cb 100%);
    }
    .football-theme {
        border-left-color: #007bff;
        background: linear-gradient(135deg, #d1ecf1 0%, #bee5eb 100%);
    }
    .metric-container {
        text-align: center;
        padding: 1rem;
        background: white;
        border-radius: 8px;
        margin: 0.5rem;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    .search-results {
        animation: fadeIn 0.5s ease-in;
    }
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(20px); }
        to { opacity: 1; transform: translateY(0); }
    }
    .loading-spinner {
        border: 4px solid #f3f3f3;
        border-top: 4px solid #3498db;
        border-radius: 50%;
        width: 50px;
        height: 50px;
        animation: spin 2s linear infinite;
        margin: 20px auto;
    }
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """, unsafe_allow_html=True)

class AdvancedWebDriver:
    """Advanced WebDriver setup with better error handling"""
    def __init__(self):
        self.driver = None
        self.setup_driver()

    def setup_driver(self):
        """Setup Chrome driver with optimal options"""
        try:
            chrome_options = Options()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--window-size=1920,1080')
            chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
            chrome_options.add_experimental_option('useAutomationExtension', False)

            self.driver = webdriver.Chrome(options=chrome_options)
            self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            return True
        except Exception as e:
            st.error(f"Failed to setup WebDriver: {str(e)}")
            return False

    def get_page(self, url, wait_time=10):
        """Get page with error handling"""
        try:
            self.driver.get(url)
            WebDriverWait(self.driver, wait_time).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            return self.driver.page_source
        except TimeoutException:
            st.warning(f"Timeout loading page: {url}")
            return None
        except Exception as e:
            st.error(f"Error loading page {url}: {str(e)}")
            return None

    def close(self):
        """Close driver safely"""
        if self.driver:
            try:
                self.driver.quit()
            except:
                pass

class CricketScraper:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
        })
        self.base_urls = {
            'espncricinfo': 'https://www.espncricinfo.com',
            'cricbuzz': 'https://www.cricbuzz.com'
        }

    def search_player(self, player_name):
        """Search for cricket player with real data scraping"""
        try:
            # Try multiple sources
            player_data = self.scrape_from_espncricinfo(player_name)
            if not player_data:
                player_data = self.scrape_from_cricbuzz(player_name)

            return player_data if player_data else self.get_fallback_cricket_data(player_name)
        except Exception as e:
            st.error(f"Error searching for cricket player: {str(e)}")
            return self.get_fallback_cricket_data(player_name)

    def scrape_from_espncricinfo(self, player_name):
        """Scrape cricket data from ESPNCricinfo"""
        try:
            # Search for player
            search_url = f"https://www.espncricinfo.com/ci/engine/match/search.html?search={quote(player_name)}"

            driver = AdvancedWebDriver()
            if not driver.driver:
                return None

            page_source = driver.get_page(search_url)
            if not page_source:
                driver.close()
                return None

            soup = BeautifulSoup(page_source, 'html.parser')

            # Look for player profile links
            player_links = soup.find_all('a', href=re.compile(r'/ci/content/player/\d+\.html'))

            if player_links:
                # Get the first player link
                player_url = urljoin(self.base_urls['espncricinfo'], player_links[0]['href'])
                player_data = self.extract_cricinfo_player_data(driver, player_url)
                driver.close()
                return player_data

            driver.close()
            return None

        except Exception as e:
            st.warning(f"ESPNCricinfo scraping failed: {str(e)}")
            return None

    def extract_cricinfo_player_data(self, driver, player_url):
        """Extract player data from Cricinfo player page"""
        try:
            page_source = driver.get_page(player_url)
            if not page_source:
                return None

            soup = BeautifulSoup(page_source, 'html.parser')

            # Extract basic info
            player_name = soup.find('h1')
            player_name = player_name.text.strip() if player_name else "Unknown"

            # Extract statistics tables
            stats_tables = soup.find_all('table', class_='engineTable')

            player_data = {
                'name': player_name,
                'country': self.extract_country_from_cricinfo(soup),
                'role': self.extract_role_from_cricinfo(soup),
                'batting_style': 'Unknown',
                'bowling_style': 'Unknown',
                'formats': {
                    'Test': {'matches': 0, 'innings': 0, 'runs': 0, 'centuries': 0, 'fifties': 0, 'highest_score': 'N/A', 'wickets': 0, 'best_bowling': 'N/A', 'debut': 'Unknown', 'average': 0, 'strike_rate': 0},
                    'ODI': {'matches': 0, 'innings': 0, 'runs': 0, 'centuries': 0, 'fifties': 0, 'highest_score': 'N/A', 'wickets': 0, 'best_bowling': 'N/A', 'debut': 'Unknown', 'average': 0, 'strike_rate': 0},
                    'T20I': {'matches': 0, 'innings': 0, 'runs': 0, 'centuries': 0, 'fifties': 0, 'highest_score': 'N/A', 'wickets': 0, 'best_bowling': 'N/A', 'debut': 'Unknown', 'average': 0, 'strike_rate': 0}
                },
                'achievements': []
            }

            # Parse statistics from tables
            for table in stats_tables:
                self.parse_cricinfo_stats_table(table, player_data)

            return player_data

        except Exception as e:
            st.warning(f"Error extracting Cricinfo data: {str(e)}")
            return None

    def extract_country_from_cricinfo(self, soup):
        """Extract country from Cricinfo page"""
        try:
            # Look for country information in various possible locations
            country_elements = soup.find_all(text=re.compile(r'(India|Australia|England|Pakistan|South Africa|New Zealand|West Indies|Sri Lanka|Bangladesh|Afghanistan|Ireland|Zimbabwe)'))
            if country_elements:
                return country_elements[0].strip()
            return "Unknown"
        except:
            return "Unknown"

    def extract_role_from_cricinfo(self, soup):
        """Extract player role from Cricinfo page"""
        try:
            # Look for role information
            role_elements = soup.find_all(text=re.compile(r'(Batsman|Bowler|All-rounder|Wicket-keeper)'))
            if role_elements:
                return role_elements[0].strip()
            return "Unknown"
        except:
            return "Unknown"

    def parse_cricinfo_stats_table(self, table, player_data):
        """Parse statistics table from Cricinfo"""
        try:
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all(['td', 'th'])
                if len(cells) >= 2:
                    # Parse different types of statistics
                    self.extract_stats_from_row(cells, player_data)
        except Exception as e:
            pass  # Continue with other tables if one fails

    def extract_stats_from_row(self, cells, player_data):
        """Extract statistics from table row"""
        try:
            # This is a simplified parser - in a real implementation,
            # you'd need to identify the specific table structure for each format
            pass
        except:
            pass

    def scrape_from_cricbuzz(self, player_name):
        """Scrape cricket data from Cricbuzz as fallback"""
        try:
            # Implement Cricbuzz scraping logic
            # This is a placeholder for the actual implementation
            return None
        except Exception as e:
            return None

    def get_fallback_cricket_data(self, player_name):
        """Fallback cricket data when scraping fails"""
        # Enhanced fallback with more players
        cricket_players = {
            'kohli': {
                'name': 'Virat Kohli',
                'country': 'India',
                'role': 'Batsman',
                'batting_style': 'Right-handed',
                'bowling_style': 'Right-arm medium',
                'formats': {
                    'Test': {
                        'matches': 111, 'innings': 180, 'runs': 8676,
                        'centuries': 29, 'fifties': 28, 'highest_score': '254*',
                        'wickets': 0, 'best_bowling': 'N/A', 'debut': '20 Jun 2011',
                        'average': 48.15, 'strike_rate': 55.78
                    },
                    'ODI': {
                        'matches': 295, 'innings': 281, 'runs': 13848,
                        'centuries': 50, 'fifties': 72, 'highest_score': '183',
                        'wickets': 4, 'best_bowling': '1/15', 'debut': '18 Aug 2008',
                        'average': 58.07, 'strike_rate': 93.17
                    },
                    'T20I': {
                        'matches': 125, 'innings': 115, 'runs': 4037,
                        'centuries': 1, 'fifties': 38, 'highest_score': '122*',
                        'wickets': 4, 'best_bowling': '1/15', 'debut': '12 Jun 2010',
                        'average': 48.69, 'strike_rate': 137.96
                    }
                },
                'achievements': ['ICC Cricketer of the Year 2017', 'ICC Test Player of the Year 2018', 'Padma Shri 2017']
            },
            'tamim': {
                'name': 'Tamim Iqbal',
                'country': 'Bangladesh',
                'role': 'Batsman',
                'batting_style': 'Left-handed',
                'bowling_style': 'N/A',
                'formats': {
                    'Test': {
                        'matches': 70, 'innings': 130, 'runs': 5134,
                        'centuries': 10, 'fifties': 31, 'highest_score': '206',
                        'wickets': 0, 'best_bowling': 'N/A', 'debut': '4 Jun 2008',
                        'average': 38.89, 'strike_rate': 55.21
                    },
                    'ODI': {
                        'matches': 243, 'innings': 240, 'runs': 8313,
                        'centuries': 14, 'fifties': 53, 'highest_score': '158',
                        'wickets': 0, 'best_bowling': 'N/A', 'debut': '9 Feb 2007',
                        'average': 36.62, 'strike_rate': 78.64
                    },
                    'T20I': {
                        'matches': 78, 'innings': 78, 'runs': 1758,
                        'centuries': 0, 'fifties': 7, 'highest_score': '103*',
                        'wickets': 0, 'best_bowling': 'N/A', 'debut': '28 Nov 2006',
                        'average': 24.08, 'strike_rate': 116.96
                    }
                },
                'achievements': [
                    'First Bangladeshi to score 10,000+ international runs',
                    'Fastest Bangladeshi to 8,000 ODI runs',
                    'Only Bangladeshi with centuries in all three formats'
                ]
            },
            'shakib': {
              'name': 'Shakib Al Hasan',
              'country': 'Bangladesh',
              'role': 'All-rounder',
              'batting_style': 'Left-handed',
              'bowling_style': 'Left-arm orthodox',
              'formats': {
                  'Test': {
                      'matches': 66, 'innings': 123, 'runs': 4454,
                      'centuries': 5, 'fifties': 31, 'highest_score': '217',
                      'wickets': 237, 'best_bowling': '7/36', 'debut': '18 May 2007',
                      'average': 39.42, 'strike_rate': 60.11
                  },
                  'ODI': {
                      'matches': 247, 'innings': 238, 'runs': 7570,
                      'centuries': 9, 'fifties': 55, 'highest_score': '134*',
                      'wickets': 317, 'best_bowling': '5/29', 'debut': '6 Aug 2006',
                      'average': 37.41, 'strike_rate': 82.73
                  },
                  'T20I': {
                      'matches': 117, 'innings': 112, 'runs': 2382,
                      'centuries': 0, 'fifties': 12, 'highest_score': '84',
                      'wickets': 140, 'best_bowling': '6/20', 'debut': '28 Nov 2006',
                      'average': 23.82, 'strike_rate': 122.60
                  }
              },
              'achievements': [
                  'ICC No. 1 All-rounder (Multiple times in Tests/ODIs)',
                  'Wisden Cricketer of the Year 2020',
                  'Only cricketer with 6000+ runs & 300+ wickets in ODIs'
              ]
          }
        }

        player_key = player_name.lower().strip()
        if player_key in cricket_players:
            return cricket_players[player_key]
        else:
            return {
                'name': player_name,
                'country': 'Unknown',
                'role': 'Unknown',
                'batting_style': 'Unknown',
                'bowling_style': 'Unknown',
                'formats': {
                    'Test': {'matches': 0, 'innings': 0, 'runs': 0, 'centuries': 0, 'fifties': 0, 'highest_score': 'N/A', 'wickets': 0, 'best_bowling': 'N/A', 'debut': 'Unknown', 'average': 0, 'strike_rate': 0},
                    'ODI': {'matches': 0, 'innings': 0, 'runs': 0, 'centuries': 0, 'fifties': 0, 'highest_score': 'N/A', 'wickets': 0, 'best_bowling': 'N/A', 'debut': 'Unknown', 'average': 0, 'strike_rate': 0},
                    'T20I': {'matches': 0, 'innings': 0, 'runs': 0, 'centuries': 0, 'fifties': 0, 'highest_score': 'N/A', 'wickets': 0, 'best_bowling': 'N/A', 'debut': 'Unknown', 'average': 0, 'strike_rate': 0}
                },
                'achievements': ['Real-time data not available - Using fallback database']
            }

class FootballScraper:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
        })
        self.base_urls = {
            'transfermarkt': 'https://www.transfermarkt.com',
            'sofascore': 'https://www.sofascore.com'
        }

    def search_player(self, player_name):
        """Search for football player with real data scraping"""
        try:
            # Try multiple sources
            player_data = self.scrape_from_transfermarkt(player_name)
            if not player_data:
                player_data = self.scrape_from_sofascore(player_name)

            return player_data if player_data else self.get_fallback_football_data(player_name)
        except Exception as e:
            st.error(f"Error searching for football player: {str(e)}")
            return self.get_fallback_football_data(player_name)

    def scrape_from_transfermarkt(self, player_name):
        """Scrape football data from Transfermarkt"""
        try:
            # Search for player
            search_url = f"https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query={quote(player_name)}"

            driver = AdvancedWebDriver()
            if not driver.driver:
                return None

            page_source = driver.get_page(search_url)
            if not page_source:
                driver.close()
                return None

            soup = BeautifulSoup(page_source, 'html.parser')

            # Look for player profile links
            player_links = soup.find_all('a', href=re.compile(r'/profil/spieler/\d+'))

            if player_links:
                # Get the first player link
                player_url = urljoin(self.base_urls['transfermarkt'], player_links[0]['href'])
                player_data = self.extract_transfermarkt_player_data(driver, player_url)
                driver.close()
                return player_data

            driver.close()
            return None

        except Exception as e:
            st.warning(f"Transfermarkt scraping failed: {str(e)}")
            return None

    def extract_transfermarkt_player_data(self, driver, player_url):
        """Extract player data from Transfermarkt player page"""
        try:
            page_source = driver.get_page(player_url)
            if not page_source:
                return None

            soup = BeautifulSoup(page_source, 'html.parser')

            # Extract basic info
            player_name = soup.find('h1')
            player_name = player_name.text.strip() if player_name else "Unknown"

            player_data = {
                'name': player_name,
                'country': self.extract_country_from_transfermarkt(soup),
                'position': self.extract_position_from_transfermarkt(soup),
                'date_of_birth': self.extract_dob_from_transfermarkt(soup),
                'club_career': {
                    'total_matches': 0,
                    'total_goals': 0,
                    'total_assists': 0,
                    'clubs': [],
                    'titles': []
                },
                'international_career': {
                    'national_team': 'Unknown',
                    'matches': 0,
                    'goals': 0,
                    'assists': 0,
                    'trophies': []
                },
                'individual_awards': []
            }

            # Extract statistics
            self.extract_transfermarkt_stats(soup, player_data)

            return player_data

        except Exception as e:
            st.warning(f"Error extracting Transfermarkt data: {str(e)}")
            return None

    def extract_country_from_transfermarkt(self, soup):
        """Extract country from Transfermarkt page"""
        try:
            country_elements = soup.find_all('img', src=re.compile(r'/flaggen/'))
            if country_elements:
                # Extract country from flag image alt text or nearby text
                return "Unknown"  # Simplified for now
            return "Unknown"
        except:
            return "Unknown"

    def extract_position_from_transfermarkt(self, soup):
        """Extract position from Transfermarkt page"""
        try:
            # Look for position information
            return "Unknown"  # Simplified for now
        except:
            return "Unknown"

    def extract_dob_from_transfermarkt(self, soup):
        """Extract date of birth from Transfermarkt page"""
        try:
            # Look for date of birth information
            return "Unknown"  # Simplified for now
        except:
            return "Unknown"

    def extract_transfermarkt_stats(self, soup, player_data):
        """Extract statistics from Transfermarkt page"""
        try:
            # Parse statistics tables
            # This would need to be implemented based on Transfermarkt's structure
            pass
        except:
            pass

    def scrape_from_sofascore(self, player_name):
        """Scrape football data from SofaScore as fallback"""
        try:
            # Implement SofaScore scraping logic
            return None
        except Exception as e:
            return None

    def get_fallback_football_data(self, player_name):
        """Fallback football data when scraping fails"""
        # Enhanced fallback with more players

        player_key = player_name.lower().strip()
        if player_key in football_players:
            return football_players[player_key]
        else:
            return {
                'name': player_name,
                'country': 'Unknown',
                'position': 'Unknown',
                'date_of_birth': 'Unknown',
                'club_career': {
                    'total_matches': 0,
                    'total_goals': 0,
                    'total_assists': 0,
                    'clubs': ['Unknown'],
                    'titles': ['Real-time data not available - Using fallback database']
                },
                'international_career': {
                    'national_team': 'Unknown',
                    'matches': 0,
                    'goals': 0,
                    'assists': 0,
                    'trophies': []
                },
                'individual_awards': ['Real-time data not available - Using fallback database']
            }

class DataRenderer:
    """Render player data with beautiful UI"""

    @staticmethod
    def render_cricket_player(player_data):
        """Render cricket player data"""
        st.markdown('<div class="search-results">', unsafe_allow_html=True)

        # Player header
        st.markdown(f'<div class="player-card cricket-theme">', unsafe_allow_html=True)
        st.markdown(f"### 🏏 {player_data['name']}")

        # Basic info
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.markdown(f"**Country:** {player_data['country']}")
        with col2:
            st.markdown(f"**Role:** {player_data['role']}")
        with col3:
            st.markdown(f"**Batting:** {player_data['batting_style']}")
        with col4:
            st.markdown(f"**Bowling:** {player_data['bowling_style']}")

        st.markdown('</div>', unsafe_allow_html=True)

        # Format tabs
        tab1, tab2, tab3 = st.tabs(["🏆 Test Cricket", "🌍 ODI Cricket", "⚡ T20I Cricket"])

        with tab1:
            DataRenderer.render_cricket_format_stats(player_data['formats']['Test'], "Test")

        with tab2:
            DataRenderer.render_cricket_format_stats(player_data['formats']['ODI'], "ODI")

        with tab3:
            DataRenderer.render_cricket_format_stats(player_data['formats']['T20I'], "T20I")

        # Achievements
        st.markdown('<div class="stat-box">', unsafe_allow_html=True)
        st.markdown("### 🏆 Achievements")
        for achievement in player_data['achievements']:
            st.markdown(f"• {achievement}")
        st.markdown('</div>', unsafe_allow_html=True)

        st.markdown('</div>', unsafe_allow_html=True)

    @staticmethod
    def render_cricket_format_stats(format_data, format_name):
        """Render statistics for a cricket format"""
        st.markdown('<div class="stat-box">', unsafe_allow_html=True)

        # Batting stats
        st.markdown(f"#### 🏏 {format_name} Batting Statistics")

        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("Matches", format_data['matches'])
            st.metric("Innings", format_data['innings'])
        with col2:
            st.metric("Runs", format_data['runs'])
            st.metric("Average", format_data['average'])
        with col3:
            st.metric("Centuries", format_data['centuries'])
            st.metric("Fifties", format_data['fifties'])
        with col4:
            st.metric("Highest Score", format_data['highest_score'])
            st.metric("Strike Rate", format_data['strike_rate'])

        # Bowling stats (if applicable)
        if format_data['wickets'] > 0:
            st.markdown(f"#### ⚾ {format_name} Bowling Statistics")
            col1, col2 = st.columns(2)
            with col1:
                st.metric("Wickets", format_data['wickets'])
            with col2:
                st.metric("Best Bowling", format_data['best_bowling'])

        # Debut info
        st.markdown(f"**Debut:** {format_data['debut']}")

        st.markdown('</div>', unsafe_allow_html=True)

    @staticmethod
    def render_football_player(player_data):
        """Render football player data"""
        st.markdown('<div class="search-results">', unsafe_allow_html=True)

        # Player header
        st.markdown(f'<div class="player-card football-theme">', unsafe_allow_html=True)
        st.markdown(f"### ⚽ {player_data['name']}")

        # Basic info
        col1, col2, col3 = st.columns(3)
        with col1:
            st.markdown(f"**Country:** {player_data['country']}")
        with col2:
            st.markdown(f"**Position:** {player_data['position']}")
        with col3:
            st.markdown(f"**Date of Birth:** {player_data['date_of_birth']}")

        st.markdown('</div>', unsafe_allow_html=True)

        # Career tabs
        tab1, tab2, tab3 = st.tabs(["🏟️ Club Career", "🌍 International Career", "🏆 Individual Awards"])

        with tab1:
            DataRenderer.render_club_career(player_data['club_career'])

        with tab2:
            DataRenderer.render_international_career(player_data['international_career'])

        with tab3:
            DataRenderer.render_individual_awards(player_data['individual_awards'])

        st.markdown('</div>', unsafe_allow_html=True)

    @staticmethod
    def render_club_career(club_data):
        """Render club career statistics"""
        st.markdown('<div class="stat-box">', unsafe_allow_html=True)

        # Overall stats
        st.markdown("#### 📊 Overall Club Statistics")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Total Matches", club_data['total_matches'])
        with col2:
            st.metric("Total Goals", club_data['total_goals'])
        with col3:
            st.metric("Total Assists", club_data['total_assists'])

        # Clubs
        st.markdown("#### 🏟️ Clubs")
        for club in club_data['clubs']:
            st.markdown(f"• {club}")

        # Titles
        st.markdown("#### 🏆 Club Titles")
        for title in club_data['titles']:
            st.markdown(f"• {title}")

        st.markdown('</div>', unsafe_allow_html=True)

    @staticmethod
    def render_international_career(intl_data):
        """Render international career statistics"""
        st.markdown('<div class="stat-box">', unsafe_allow_html=True)

        st.markdown(f"#### 🌍 {intl_data['national_team']} National Team")

        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Matches", intl_data['matches'])
        with col2:
            st.metric("Goals", intl_data['goals'])
        with col3:
            st.metric("Assists", intl_data['assists'])

        # Trophies
        if intl_data['trophies']:
            st.markdown("#### 🏆 International Trophies")
            for trophy in intl_data['trophies']:
                st.markdown(f"• {trophy}")

        st.markdown('</div>', unsafe_allow_html=True)

    @staticmethod
    def render_individual_awards(awards):
        """Render individual awards"""
        st.markdown('<div class="stat-box">', unsafe_allow_html=True)
        st.markdown("#### 🏆 Individual Awards & Recognition")
        for award in awards:
            st.markdown(f"• {award}")
        st.markdown('</div>', unsafe_allow_html=True)

class SearchEngine:
    """Main search engine coordinator"""

    def __init__(self):
        self.cricket_scraper = CricketScraper()
        self.football_scraper = FootballScraper()

    def search(self, sport, player_name):
        """Search for player based on sport"""
        if sport.lower() == 'cricket':
            return self.cricket_scraper.search_player(player_name)
        elif sport.lower() == 'football':
            return self.football_scraper.search_player(player_name)
        else:
            return None

def show_loading_animation():
    """Show loading animation"""
    st.markdown('<div class="loading-spinner"></div>', unsafe_allow_html=True)
    st.markdown("**Searching for player data...**")

def main():
    """Main Streamlit application"""
    st.set_page_config(
        page_title="Enhanced Sports Search Engine",
        page_icon="🏆",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    # Load custom CSS
    load_css()

    # Main header
    st.markdown('<h1 class="main-header">🏆 Enhanced Sports Search Engine</h1>', unsafe_allow_html=True)
    st.markdown("---")

    # Sidebar
    with st.sidebar:
        st.markdown("### 🔍 Search Configuration")

        # Sport selection
        sport = st.selectbox(
            "Select Sport:",
            ["Cricket", "Football"],
            key="sport_selector"
        )

        # Player name input
        player_name = st.text_input(
            "Enter Player Name:",
            placeholder="e.g., Virat Kohli, Lionel Messi",
            key="player_input"
        )

        # Search button
        search_clicked = st.button("🔍 Search Player", type="primary", use_container_width=True)

        # Information
        st.markdown("---")
        st.markdown("### ℹ️ Information")
        st.markdown("""
        **Supported Sports:**
        - 🏏 Cricket (Test, ODI, T20I)
        - ⚽ Football (Club & International)

        **Data Sources:**
        - ESPNCricinfo
        - Cricbuzz
        - Transfermarkt
        - SofaScore
        """)

        # Sample searches
        st.markdown("### 💡 Try These Players")
        sample_players = {
            "Cricket": ["Virat Kohli", "MS Dhoni", "Rohit Sharma"],
            "Football": ["Lionel Messi", "Cristiano Ronaldo", "Kylian Mbappe"]
        }

        for sample_player in sample_players[sport]:
            if st.button(f"🔗 {sample_player}", key=f"sample_{sample_player}"):
                st.session_state.player_input = sample_player
                st.rerun()

    # Main content area
    if search_clicked and player_name:
        # Show loading animation
        loading_placeholder = st.empty()
        with loading_placeholder.container():
            show_loading_animation()

        # Initialize search engine
        search_engine = SearchEngine()

        try:
            # Perform search
            player_data = search_engine.search(sport, player_name)

            # Clear loading animation
            loading_placeholder.empty()

            if player_data:
                # Render results based on sport
                if sport.lower() == 'cricket':
                    DataRenderer.render_cricket_player(player_data)
                elif sport.lower() == 'football':
                    DataRenderer.render_football_player(player_data)

                # Success message
                st.success(f"✅ Successfully found data for {player_data['name']}!")

                # Export options
                st.markdown("---")
                col1, col2 = st.columns(2)

                with col1:
                    if st.button("📊 Export to JSON"):
                        json_data = json.dumps(player_data, indent=2)
                        st.download_button(
                            label="💾 Download JSON",
                            data=json_data,
                            file_name=f"{player_data['name'].replace(' ', '_')}_stats.json",
                            mime="application/json"
                        )

                with col2:
                    if st.button("📋 Copy Player Info"):
                        player_info = f"Player: {player_data['name']}\nCountry: {player_data.get('country', 'Unknown')}\nSport: {sport}"
                        st.code(player_info)

            else:
                st.error(f"❌ No data found for '{player_name}' in {sport}")
                st.info("💡 Try checking the spelling or try a different player name.")

        except Exception as e:
            loading_placeholder.empty()
            st.error(f"⚠️ An error occurred while searching: {str(e)}")
            st.info("🔄 Please try again or contact support if the issue persists.")

    elif search_clicked and not player_name:
        st.warning("⚠️ Please enter a player name to search.")

    else:
        # Welcome message
        st.markdown("""
        <div style="text-align: center; padding: 3rem;">
            <h2>🔍 Welcome to Enhanced Sports Search!</h2>
            <p style="font-size: 1.2rem; color: #666;">
                Search for your favorite cricket and football players to get comprehensive statistics,
                career information, and achievements.
            </p>
            <p style="color: #888;">
                Select a sport from the sidebar and enter a player name to get started.
            </p>
        </div>
        """, unsafe_allow_html=True)

        # Featured players showcase
        st.markdown("### 🌟 Featured Players")

        col1, col2 = st.columns(2)

        with col1:
            st.markdown("""
            **🏏 Cricket Legends:**
            - Virat Kohli (India)
            - MS Dhoni (India)
            - Rohit Sharma (India)
            """)

        with col2:
            st.markdown("""
            **⚽ Football Stars:**
            - Lionel Messi (Argentina)
            - Cristiano Ronaldo (Portugal)
            - Kylian Mbappé (France)
            """)

if __name__ == "__main__":
    main()

Writing enhanced_sports_search_app.py


In [5]:
import subprocess
import threading
import time
from pyngrok import ngrok, conf
import os

In [8]:
# Set ngrok configuration (optional: add your authtoken)
# Get free authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2vwDgSqFV5DpwG9AeVJiljdvrYL_32CShYkruUBYp13vasVpZ")

In [9]:
def run_streamlit():
    """Function to run the Streamlit app"""
    subprocess.run([
        "streamlit", "run", "enhanced_sports_search_app.py",
        "--server.port", "8501",
        "--server.address", "0.0.0.0",
        "--server.headless", "true",
        "--server.fileWatcherType", "none",
        "--browser.gatherUsageStats", "false"
    ])

In [ ]:
def start_app_with_tunnel():
    """Start Streamlit app with ngrok tunnel"""

    print("🚀 Starting Enhanced Sports Search Engine...")
    print("=" * 60)

    # Start ngrok tunnel
    try:
        # Kill any existing ngrok processes
        ngrok.kill()

        # Create tunnel
        public_url = ngrok.connect(8501, "http")

        print(f"🌐 Ngrok tunnel created successfully!")
        print(f"📱 Public URL: {public_url}")
        print(f"🔗 Access your app at: {public_url}")
        print("=" * 60)
        print("⚠️  IMPORTANT: Keep this cell running to maintain the tunnel!")
        print("=" * 60)

        # Start Streamlit in a separate thread
        streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
        streamlit_thread.start()

        # Keep the tunnel alive
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("\n🛑 Stopping the application...")
            ngrok.kill()

    except Exception as e:
        print(f"❌ Error starting tunnel: {str(e)}")
        print("💡 Try restarting the runtime and running the setup again")

# Start the application
start_app_with_tunnel()

🚀 Starting Enhanced Sports Search Engine...
🌐 Ngrok tunnel created successfully!
📱 Public URL: NgrokTunnel: "https://2413117549a2.ngrok-free.app" -> "http://localhost:8501"
🔗 Access your app at: NgrokTunnel: "https://2413117549a2.ngrok-free.app" -> "http://localhost:8501"
⚠️  IMPORTANT: Keep this cell running to maintain the tunnel!
